# IS 4487 Lab 11

## Learning Objective

Use Linear Regression to predict the AQI in Utah.

## Outline

- Pull the latest "Daily AQI by County" file from this link: https://aqs.epa.gov/aqsweb/airdata/download_files.html#AQI

- Your target variable will be *AQI", which is the value of the air quality index

- We will focus the analysis on only the air quality in the state of Utah.  

- Note that there is a several-month lag in preparing data; you should check to see if your file has a full year of data from January to December.  If not, use the previous year.    

- The AQI is divided into six categories:

*Air Quality Index*

|(AQI) Values	|Levels of Health Concern	        |
|---------------|--------|
|0-50	        |Good	 |
|51-100	        |Moderate	 |
|101-150	    |Unhealthy for Sensitive Groups	|
|151 to 200	    |Unhealthy	 |
|201 to 300	    |Very Unhealthy	 |
|301 to 500	    |Hazardous	 |

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Labs/Scripts/lab_11_air_quality_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Libraries

➡️ Assignment Tasks
- Load any necessary libraries

In [1]:
import pandas as pd
import matplotlib as mpl
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier, export_graphviz # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics  #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report

## Import Data into Dataframe

➡️ Assignment Tasks
- Pull the latest full year of data using the "Daily AQI by County" files from this link: https://aqs.epa.gov/aqsweb/airdata/download_files.html#AQI
- Make sure to UNZIP the file
- Import data from the air quality dataset into a dataframe
- Describe or profile the dataframe

In [9]:
# import the data set
df = pd.read_csv('https://raw.githubusercontent.com/tired24/IS-4487-/main/daily_aqi_by_county_2024.csv')


Check whether the dataset was uploaded successfully

In [10]:
# Top 5 Rows in dataset
df.head()

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,Alabama,Baldwin,1,3,2024-01-03,41,Good,PM2.5,01-003-0010,1
1,Alabama,Baldwin,1,3,2024-01-04,38,Good,PM2.5,01-003-0010,1
2,Alabama,Baldwin,1,3,2024-01-05,44,Good,PM2.5,01-003-0010,1
3,Alabama,Baldwin,1,3,2024-01-06,7,Good,PM2.5,01-003-0010,1
4,Alabama,Baldwin,1,3,2024-01-07,29,Good,PM2.5,01-003-0010,1


In [11]:
# Last 5 rows in data
df.tail()

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
206914,Wyoming,Weston,56,45,2024-06-26,45,Good,Ozone,56-045-0003,1
206915,Wyoming,Weston,56,45,2024-06-27,48,Good,Ozone,56-045-0003,1
206916,Wyoming,Weston,56,45,2024-06-28,47,Good,Ozone,56-045-0003,1
206917,Wyoming,Weston,56,45,2024-06-29,37,Good,Ozone,56-045-0003,1
206918,Wyoming,Weston,56,45,2024-06-30,47,Good,Ozone,56-045-0003,1


Check whether the time frame covers 12 month period
- Dataset: daily_aqi_by_country_2024
- Pull the latest full year of data using the "Daily AQI by County" files from this link
- Note that there is a several-month lag in preparing data; you should check to see if your file has a full year of data from January to December. If not, use the previous year.

In [19]:
# Create a copy of df
# We do not what to alter original dataset (atleast for now)
# Creating a copy of df is one way to check the timeline listed within the df
df_check = df.copy()

In [20]:
# Create "Year" & "Month" columns to later filter out the timeframe presented in df
df_check['Year'] = df_check['Date'].dt.year
df_check['Month'] = df_check['Date'].dt.month

In [21]:
# Check distinct Year-Month combinations
df_check[['Year', 'Month']].drop_duplicates().sort_values(['Year', 'Month'])

,Year,Month
0,2024,1
29,2024,2
58,2024,3
89,2024,4
119,2024,5
149,2024,6
179,2024,7
635,2024,8
3881,2024,9
5275,2024,10


Describe or profile the dataframe

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206919 entries, 0 to 206918
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   State Name                 206919 non-null  object        
 1   county Name                206919 non-null  object        
 2   State Code                 206919 non-null  int64         
 3   County Code                206919 non-null  int64         
 4   Date                       206919 non-null  datetime64[ns]
 5   AQI                        206919 non-null  int64         
 6   Category                   206919 non-null  object        
 7   Defining Parameter         206919 non-null  object        
 8   Defining Site              206919 non-null  object        
 9   Number of Sites Reporting  206919 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 15.8+ MB


In [23]:
df.describe()

,State Code,County Code,Date,AQI,Number of Sites Reporting
count,206919.000000,206919.000000,206919,206919.000000,206919.000000
mean,30.068205,80.430077,2024-05-03 06:47:20.311426048,42.051513,1.898095
min,1.000000,1.000000,2024-01-01 00:00:00,0.000000,1.000000
25%,17.000000,23.000000,2024-03-03 00:00:00,32.000000,1.000000
50%,30.000000,59.000000,2024-04-29 00:00:00,41.000000,1.000000
75%,42.000000,107.000000,2024-06-25 00:00:00,50.000000,2.000000
max,80.000000,810.000000,2024-11-01 00:00:00,1322.000000,33.000000
std,15.845597,92.711027,NaN,19.166162,2.279127


## Prepare Data

➡️ Assignment Tasks
- Filter the data to use Utah data only
- Create one dummy variable (true/false) for each of the Defining Parameter values    
- Create variables for month of year, year, and season
- Perform any other data cleanup needed (remove outliers, nulls, etc.)
- After filtering for Utah, remove the geographical variables that remain (county, state) since those non-numeric values can't be used.  Remove any other non-numeric variables.
- Select the data you would like to use in the model.  If you aggregate data, you will have to decide whether to use the min, max or mean value for AQI
- Split the data 80/20 for training and testing

In [25]:
#filter for Utah data only
df_utah = df[df['State Name'] == 'Utah']
df_utah.head()

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
178824,Utah,Box Elder,49,3,2024-01-01,71,Moderate,PM2.5,49-003-0005,2
178825,Utah,Box Elder,49,3,2024-01-02,64,Moderate,PM2.5,49-003-0005,2
178826,Utah,Box Elder,49,3,2024-01-03,66,Moderate,PM2.5,49-003-0005,2
178827,Utah,Box Elder,49,3,2024-01-04,60,Moderate,PM2.5,49-003-0005,2
178828,Utah,Box Elder,49,3,2024-01-05,29,Good,Ozone,49-003-7001,2


In [27]:
# Create one dummy variable (true/false) for each of the Defining Parameter values
df_utah = pd.get_dummies(df_utah, columns=['Defining Parameter'])
df_utah.head()

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Site,Number of Sites Reporting,Defining Parameter_CO,Defining Parameter_NO2,Defining Parameter_Ozone,Defining Parameter_PM10,Defining Parameter_PM2.5
178824,Utah,Box Elder,49,3,2024-01-01,71,Moderate,49-003-0005,2,False,False,False,False,True
178825,Utah,Box Elder,49,3,2024-01-02,64,Moderate,49-003-0005,2,False,False,False,False,True
178826,Utah,Box Elder,49,3,2024-01-03,66,Moderate,49-003-0005,2,False,False,False,False,True
178827,Utah,Box Elder,49,3,2024-01-04,60,Moderate,49-003-0005,2,False,False,False,False,True
178828,Utah,Box Elder,49,3,2024-01-05,29,Good,49-003-7001,2,False,False,True,False,False


In [30]:
#create columns / create variables for month of year, year, and season
df_utah['Year'] = df_utah['Date'].dt.year
df_utah['Month'] = df_utah['Date'].dt.month

# Move "Year" & "Month" column right after "Date"
year_column = df_utah.pop('Year')
month_column = df_utah.pop('Month')
date_idx = df_utah.columns.get_loc('Date')
df_utah.insert(date_idx + 1, 'Year', year_column, allow_duplicates=True)
df_utah.insert(date_idx + 2, 'Month', month_column, allow_duplicates=True)

# Display the updated df
df_utah.head()

,State Name,county Name,State Code,County Code,Date,Year,Month,AQI,Category,Defining Site,Number of Sites Reporting,Defining Parameter_CO,Defining Parameter_NO2,Defining Parameter_Ozone,Defining Parameter_PM10,Defining Parameter_PM2.5
178824,Utah,Box Elder,49,3,2024-01-01,2024,1,71,Moderate,49-003-0005,2,False,False,False,False,True
178825,Utah,Box Elder,49,3,2024-01-02,2024,1,64,Moderate,49-003-0005,2,False,False,False,False,True
178826,Utah,Box Elder,49,3,2024-01-03,2024,1,66,Moderate,49-003-0005,2,False,False,False,False,True
178827,Utah,Box Elder,49,3,2024-01-04,2024,1,60,Moderate,49-003-0005,2,False,False,False,False,True
178828,Utah,Box Elder,49,3,2024-01-05,2024,1,29,Good,49-003-7001,2,False,False,True,False,False


In [35]:
# create variables for season
def get_season(month):
  if month in [12,1,2]:
    return "Winter"
  elif month in [3,4,5]:
    return "Spring"
  elif month in [6,7,8]:
    return "Summer"
  elif month in [9,10,11]:
    return "Fall"
df_utah["Season"] = df_utah["Month"].apply(get_season)

# Move "Season" column right after "Month"
season_column = df_utah.pop('Season')
month_idx = df_utah.columns.get_loc('Month')
df_utah.insert(month_idx + 1, 'Season', season_column, allow_duplicates=True)

# Display the updated df
df_utah.head()

,State Name,county Name,State Code,County Code,Date,Year,Month,Season,AQI,Category,Defining Site,Number of Sites Reporting,Defining Parameter_CO,Defining Parameter_NO2,Defining Parameter_Ozone,Defining Parameter_PM10,Defining Parameter_PM2.5
178824,Utah,Box Elder,49,3,2024-01-01,2024,1,Winter,71,Moderate,49-003-0005,2,False,False,False,False,True
178825,Utah,Box Elder,49,3,2024-01-02,2024,1,Winter,64,Moderate,49-003-0005,2,False,False,False,False,True
178826,Utah,Box Elder,49,3,2024-01-03,2024,1,Winter,66,Moderate,49-003-0005,2,False,False,False,False,True
178827,Utah,Box Elder,49,3,2024-01-04,2024,1,Winter,60,Moderate,49-003-0005,2,False,False,False,False,True
178828,Utah,Box Elder,49,3,2024-01-05,2024,1,Winter,29,Good,49-003-7001,2,False,False,True,False,False


In [36]:
#data cleanup / Perform any other data cleanup needed (remove outliers, nulls, etc.)
df_utah.describe()

,State Code,County Code,Date,Year,Month,AQI,Number of Sites Reporting
count,4206.0,4206.000000,4206,4206.0,4206.000000,4206.000000,4206.000000
mean,49.0,29.262007,2024-05-27 04:49:38.601997056,2024.0,5.358298,50.285307,1.798859
min,49.0,3.000000,2024-01-01 00:00:00,2024.0,1.000000,2.000000,1.000000
25%,49.0,11.000000,2024-03-13 00:00:00,2024.0,3.000000,40.000000,1.000000
50%,49.0,35.000000,2024-05-27 00:00:00,2024.0,5.000000,46.000000,1.000000
75%,49.0,47.000000,2024-08-10 00:00:00,2024.0,8.000000,58.000000,2.000000
max,49.0,57.000000,2024-11-01 00:00:00,2024.0,11.000000,230.000000,9.000000
std,0.0,18.994181,NaN,0.0,2.817267,19.103658,2.008652


In [ ]:
# Number of missing values
no_of_missing_values = df_utah.isnull().sum()
no_of_missing_values

In [ ]:
#select final columns for use

In [ ]:
#split the data into training and testing datasets

## Create Model

➡️ Assignment Tasks
- Create a simple linear regression to predict AQI based on as many variables as you can use or derive.  (for example, sklearn LinearRegression)
- Evaluate the model by displaying the R squared value  
- Visualize the correlation between the target variable and at least one of the independent variables

In [ ]:
#create regression or classification model

In [ ]:
#print the R squared value

In [ ]:
#visual

## Make a prediction

➡️ Assignment Tasks
- What would you predict the average AQI to be in January of the upcoming year?  

In [ ]:
#predicted AQI

## OPTIONAL: Compare Air Quality

➡️ Assignment Tasks
- Download the data from several previous years using this website: https://aqs.epa.gov/aqsweb/airdata/download_files.html#AQI
- Append the new data to the previous dataframe
- Use the year as a variable in your regression.  Is year a significant factor in predicting AQI?

In [ ]:
#import, append and create new model